<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Topic_Modeling_Full_NewIter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs and imports

In [1]:
!pip install --upgrade spacy
!python -m spacy download en_core_web_lg
!pip install -U kaleido
!pip install bertopic

     |████████████████████████████████| 6.2 MB 4.5 MB/s 
     |████████████████████████████████| 181 kB 34.7 MB/s 
     |████████████████████████████████| 457 kB 39.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 653 kB 36.2 MB/s 
     |████████████████████████████████| 10.1 MB 27.2 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    U

In [2]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import re
import string

import nltk
nltk.download('stopwords')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import kaleido
nlp = spacy.load('en_core_web_lg')
multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2

## Define functions

In [3]:
def berttopic_basic(df):

  topic_model = BERTopic()
  topics, probs = topic_model.fit_transform(df)

  # added lemmatization
  # update using lemmatization

  tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())
  # vectorizer_model = CountVectorizer(stop_words="English", ngram_range=(1, 5))
  topic_model.update_topics(df, topics, vectorizer_model=tf_vectorizer)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer:
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, doc):
    return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]



import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

def create_input_new(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']

  # remove blanks
  df = df[df['Combined']!=" "]

  #keep only selected cols
  df_sel = df[['Authors','Pub Year','Research Area','Combined',"Pub TYpe"]]
  df_sel= df_sel.rename(columns={'Pub Year':'pub_year',"Research Area":"research_area","Authors":"authors","Combined":"combined","Pub TYpe":"pub_type"})

  combined = list(df_sel['combined'])

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, '', l )
    comb_clean.append(mod_string)

  # merge back to df
  df_sel['combined'] = comb_clean

  # filter spurtious yeats
  df_sel = df_sel[df_sel['pub_year']!='12.0.1.2']

  # convert years to int
  # df_sel['pub_year'] = df_sel['pub_year'].astype(str)
  df_sel['pub_year'] = df_sel['pub_year'].astype(str).replace('\.0', '', regex=True).astype(int)
  # if year is 201, that is mistyped fom 2001
  df_sel[df_sel['pub_year']==201]['pub_year'] = 2001


  return df_sel



def lemma_spacy(df_combined):

  filt_combined = []
  for word in nlp(df_combined.lower()):
    # token = word.lower()
    if word.lemma_ != '-PRON-' :
      filt_combined.append(word.lemma_)

  new_df = " ".join(filt_combined)

  return new_df  


def remove_stop_df(df_combined):

  # remove stopwords

  new_df= []
  filt_combined = []
  for word in word_tokenize(df_combined):

    if word.lower() not in stopwords.words('english'):
      # print(word)
      if word.lower() == "perovskites":
        filt_combined.append("perovskite")
      else:
        filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip

  

def flatten(t):
    return [item for sublist in t for item in sublist]


def get_authors(input_data,rep_docs):
  tt = input_data['combined'].to_list()

  auth_list= []
  for d in rep_docs:
    ind = tt.index(d)
    auth_str = input_data.authors[ind]
    auth_el = auth_str.split(" ,")
    auth_list.append(auth_el)

  auth_list = flatten(auth_list)
  if '' in auth_list:
    auth_list.remove('')

  # unduplicate repeating authors 
  # print("Length of List:",len(auth_list))
  # print(auth_list)


  final_auth_list = list(set(auth_list))
  if '' in final_auth_list:
    final_auth_list.remove('')

  # print(final_auth_list)
  # print("Length of Final List:",len(final_auth_list))

  return final_auth_list


def author_all_topics(model,input_data):

  dict_df = {}
  for i in range(len(model.topics)-1):
    rep_docs = model.get_representative_docs(i)
    author_list = get_authors(input_data,rep_docs)

    dict_df[i] = author_list

  # create df with topics and authors
  author_topics = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_df.items() ]))

  return author_topics,dict_df


def remove_punct_df(df):

  # remove punctuation
  # text_col = [''.join(letter for letter in word if letter not in string.punctuation) for word in df]

  filt_combined = []
  for word in word_tokenize(df):

    if word.lower() not in string.punctuation:
      # print(word)
      
      filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## define path of file

In [5]:
drive.mount('/content/drive')

# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

Mounted at /content/drive


['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_8.2.2.xls',
 'Beamline_

## topic modeling --  BERTopic: Mini LM

In [ ]:
start_time = time.time()
iter_version = "BERTopic_0511"


for i in range(35,len(files)):
# for i in range(1):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]
  # beam_name= "Beamline_8.3.2.xls"
  # beam_name= "Beamline_7.3.3.xls"
  

  df = pd.read_table(path + files[i], on_bad_lines='skip')
  # df = pd.read_table(path + beam_name, on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input_new(df)

  # remove stopwords
  input_data['combined'] = input_data['combined'].apply(remove_stop_df)

  # lemmatize
  input_data['combined'] = input_data['combined'].apply(lemma_spacy)
  # df_lemma = input_data['combined']

  # remove punct
  # input_data['combined'] = input_data['combined'].apply(remove_punct_df)

   # two iterations - one for unigram, other for bigram
  for i in range(2):

    print("******For ngram {}*****:".format(i+1))

    topic_model = BERTopic(nr_topics='auto',verbose=True,n_gram_range=(i+1,i+1))
    list_text = input_data['combined'].to_list()
    topics, probs = topic_model.fit_transform(list_text)

    years = input_data['pub_year'].to_list()

    # defining path
    base_path = '/content/drive/MyDrive/NLP/' + iter_version + "_ngram_" + str(i+1) + "/" 
    model_path = base_path + beam_name + "/"
    create_path_if_not_exists(model_path)

    # save model as pickle file
    file_path = model_path + "model" + beam_name + ".pkl"
    joblib.dump(topic_model, file_path) 

    # dynamic topic modeling
    topics_over_time = topic_model.topics_over_time(list_text, topics, years)
    fig_time = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)
    fig_name_png = model_path + "topic_time" + beam_name  +".png"
    fig_time.write_image(fig_name_png)
    fig_name_html = model_path + "topic_time" + beam_name  +".html"
    fig_time.write_html(fig_name_html)

    # visualize barchart of topics as html
    fig = topic_model.visualize_barchart(top_n_topics = len(topic_model.topics))
    fig_name = model_path + "bar_chart" + beam_name  +".html"
    fig_name_png = model_path + "bar_chart" + beam_name  +".png"

    fig.write_html(fig_name)
    fig.write_image(fig_name_png)


    # save as excel
    # df_info = pd.DataFrame(model.get_topic_info())
    excel_name = model_path + "Topic_Results.xlsx"
    # df_info.to_excel(excel_name,sheet_name="topic_info")

    df_topics = pd.DataFrame(topic_model.topics)
    df_topics.to_excel(excel_name,sheet_name="topic_words")

    print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))



File: 1 Beamline_1.4.xls
******For ngram 1*****:


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

2022-05-11 09:42:55,330 - BERTopic - Transformed documents to Embeddings
2022-05-11 09:42:59,147 - BERTopic - Reduced dimensionality
2022-05-11 09:42:59,175 - BERTopic - Clustered reduced embeddings
2022-05-11 09:42:59,402 - BERTopic - Reduced number of topics from 9 to 6
27it [00:00, 43.11it/s]


File: 1 done 
******For ngram 2*****:


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

...

2022-05-11 10:08:11,891 - BERTopic - Transformed documents to Embeddings
2022-05-11 10:08:16,590 - BERTopic - Reduced dimensionality
2022-05-11 10:08:16,630 - BERTopic - Clustered reduced embeddings
2022-05-11 10:08:16,964 - BERTopic - Reduced number of topics from 17 to 17
30it [00:00, 36.47it/s]


File: 1 done 
******For ngram 2*****:


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2022-05-11 10:08:24,100 - BERTopic - Transformed documents to Embeddings
2022-05-11 10:08:28,861 - BERTopic - Reduced dimensionality
2022-05-11 10:08:28,899 - BERTopic - Clustered reduced embeddings
2022-05-11 10:08:29,402 - BERTopic - Reduced number of topics from 13 to 5
30it [00:01, 15.87it/s]


File: 2 done 
File: 36 Beamline_7.0.1.xls


In [6]:
start_time = time.time()
iter_version = "BERTopic_0511"


for i in range(35,len(files)):
# for i in range(1):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]
  # beam_name= "Beamline_8.3.2.xls"
  # beam_name= "Beamline_7.3.3.xls"
  

  df = pd.read_table(path + files[i], on_bad_lines='skip')
  # df = pd.read_table(path + beam_name, on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input_new(df)

  # remove stopwords
  input_data['combined'] = input_data['combined'].apply(remove_stop_df)

  # lemmatize
  input_data['combined'] = input_data['combined'].apply(lemma_spacy)
  # df_lemma = input_data['combined']

  # remove punct
  # input_data['combined'] = input_data['combined'].apply(remove_punct_df)

   # two iterations - one for unigram, other for bigram
  for i in range(2):

    print("******For ngram {}*****:".format(i+1))

    topic_model = BERTopic(nr_topics='auto',verbose=True,n_gram_range=(i+1,i+1))
    list_text = input_data['combined'].to_list()
    topics, probs = topic_model.fit_transform(list_text)

    years = input_data['pub_year'].to_list()

    # defining path
    base_path = '/content/drive/MyDrive/NLP/' + iter_version + "_ngram_" + str(i+1) + "/" 
    model_path = base_path + beam_name + "/"
    create_path_if_not_exists(model_path)

    # save model as pickle file
    file_path = model_path + "model" + beam_name + ".pkl"
    joblib.dump(topic_model, file_path) 

    # dynamic topic modeling
    topics_over_time = topic_model.topics_over_time(list_text, topics, years)
    fig_time = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)
    fig_name_png = model_path + "topic_time" + beam_name  +".png"
    fig_time.write_image(fig_name_png)
    fig_name_html = model_path + "topic_time" + beam_name  +".html"
    fig_time.write_html(fig_name_html)

    # visualize barchart of topics as html
    fig = topic_model.visualize_barchart(top_n_topics = len(topic_model.topics))
    fig_name = model_path + "bar_chart" + beam_name  +".html"
    fig_name_png = model_path + "bar_chart" + beam_name  +".png"

    fig.write_html(fig_name)
    fig.write_image(fig_name_png)


    # save as excel
    # df_info = pd.DataFrame(model.get_topic_info())
    excel_name = model_path + "Topic_Results.xlsx"
    # df_info.to_excel(excel_name,sheet_name="topic_info")

    df_topics = pd.DataFrame(topic_model.topics)
    df_topics.to_excel(excel_name,sheet_name="topic_words")

    print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))



File: 36 Beamline_7.0.1.xls
******For ngram 1*****:


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

2022-05-11 10:50:45,793 - BERTopic - Transformed documents to Embeddings
2022-05-11 10:50:49,661 - BERTopic - Reduced dimensionality
2022-05-11 10:50:49,687 - BERTopic - Clustered reduced embeddings
2022-05-11 10:50:49,811 - BERTopic - Reduced number of topics from 3 to 3
16it [00:00, 66.12it/s]


File: 1 done 
******For ngram 2*****:


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

2022-05-11 10:50:54,412 - BERTopic - Transformed documents to Embeddings
2022-05-11 10:50:58,717 - BERTopic - Reduced dimensionality
2022-05-11 10:50:58,742 - BERTopic - Clustered reduced embeddings
2022-05-11 10:50:58,950 - BERTopic - Reduced number of topics from 3 to 3
16it [00:00, 31.18it/s]


File: 2 done 
Total time taken (mins):  13


In [ ]:
input_data['combined'].to_csv("final_input.csv")

In [ ]:
df_lemma.to_csv("after_lemma.csv")

In [ ]:
lemma_spacy("Mesoscopic construct order orient metal Organic frameworks Plasmonic Silver Nanocrystals enclose octahedral silver nanocrystal Ag NCs metal organic framework mof make mesoscopic construct oh nano ag‚äÇmof interface Ag mof pristine MOF order crystalline orient Ag NCs achieve atomic layer deposition aluminum oxide Ag ncs addition tetra topic porphyrin base linker 4,4‚Ä≤,4‚Ä≥,4 ‚åÑ porphyrin-5,10,15,20 tetrayl tetrabenzoic acid h4tcpp react alumina make MOF al2 oh 2tcpp enclosure around Ag ncs alumina thickness precisely control 0.1 3 nm thus allow control MOF thickness 10 50 nm electron microscopy grazing angle X ray diffraction confirm order orientation MOF virtue porphyrin unit perpendicular plane Ag use surface enhance Raman spectroscopy directly track metalation process porphyrin map distribution metalate unmetalated linker single nanoparticle level")

'mesoscopic construct order orient metal organic framework Plasmonic Silver Nanocrystals enclose octahedral silver nanocrystal Ag NCs metal organic framework mof make mesoscopic construct oh nano ag‚äçmof interface Ag mof pristine MOF order crystalline orient Ag NCs achieve atomic layer deposition aluminum oxide Ag ncs addition tetra topic porphyrin base linker 4,4‚ä≤,4‚ä≥,4 ‚ åÑ porphyrin-5,10,15,20 tetrayl tetrabenzoic acid h4tcpp react alumina make MOF al2 oh 2tcpp enclosure around Ag ncs alumina thickness precisely control 0.1 3 nm thus allow control mof thickness 10 50 nm electron microscopy grazing angle X ray diffraction confirm order orientation MOF virtue porphyrin unit perpendicular plane Ag use surface enhance Raman spectroscopy directly track metalation process porphyrin map distribution metalate unmetalated linker single nanoparticle level'

## SPECTER

In [7]:
start_time = time.time()
iter_version = "BERTopic_SPECTER_0511"


for i in range(len(files)):
# for i in range(1):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]
  # beam_name= "Beamline_8.3.2.xls"
  # beam_name= "Beamline_7.3.3.xls"
  

  df = pd.read_table(path + files[i], on_bad_lines='skip')
  # df = pd.read_table(path + beam_name, on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input_new(df)

  # remove stopwords
  input_data['combined'] = input_data['combined'].apply(remove_stop_df)

  # lemmatize
  input_data['combined'] = input_data['combined'].apply(lemma_spacy)
  # df_lemma = input_data['combined']

  # remove punct
  input_data['combined'] = input_data['combined'].apply(remove_punct_df)

   # two iterations - one for unigram, other for bigram
  for j in range(2):

    print("******For ngram {}*****:".format(j+1))

    sentence_model = SentenceTransformer('allenai-specter')
    topic_model = BERTopic(embedding_model=sentence_model,nr_topics='auto',verbose=True,n_gram_range=(j+1,j+1))

    # topic_model = BERTopic(nr_topics='auto',verbose=True,n_gram_range=(i+1,i+1))
    list_text = input_data['combined'].to_list()

    try:
      topics, probs = topic_model.fit_transform(list_text)

      years = input_data['pub_year'].to_list()

      # defining path
      base_path = '/content/drive/MyDrive/NLP/' + iter_version + "_ngram_" + str(j+1) + "/" 
      model_path = base_path + beam_name + "/"
      create_path_if_not_exists(model_path)

      # save model as pickle file
      file_path = model_path + "model" + beam_name + ".pkl"
      joblib.dump(topic_model, file_path) 

      # dynamic topic modeling
      topics_over_time = topic_model.topics_over_time(list_text, topics, years)
      fig_time = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)
      fig_name_png = model_path + "topic_time" + beam_name  +".png"
      fig_time.write_image(fig_name_png)
      fig_name_html = model_path + "topic_time" + beam_name  +".html"
      fig_time.write_html(fig_name_html)

      # visualize barchart of topics as html
      fig = topic_model.visualize_barchart(top_n_topics = len(topic_model.topics))
      fig_name = model_path + "bar_chart" + beam_name  +".html"
      fig_name_png = model_path + "bar_chart" + beam_name  +".png"

      fig.write_html(fig_name)
      fig.write_image(fig_name_png)


      # save as excel
      # df_info = pd.DataFrame(model.get_topic_info())
      excel_name = model_path + "Topic_Results.xlsx"
      # df_info.to_excel(excel_name,sheet_name="topic_info")

      df_topics = pd.DataFrame(topic_model.topics)
      df_topics.to_excel(excel_name,sheet_name="topic_words")

    except:
      print("*****Some Error*****:")

    print("File: {} done ".format(j+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))



File: 1 Beamline_1.4.xls
******For ngram 1*****:


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

...

2022-05-11 11:14:05,128 - BERTopic - Transformed documents to Embeddings
2022-05-11 11:14:10,681 - BERTopic - Reduced dimensionality
2022-05-11 11:14:10,721 - BERTopic - Clustered reduced embeddings
2022-05-11 11:14:11,667 - BERTopic - Reduced number of topics from 15 to 15
26it [00:01, 13.19it/s]


File: 2 done 
File: 21 Beamline_4.0.3.1.xls
******For ngram 1*****:


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2022-05-11 11:14:23,898 - BERTopic - Transformed documents to Embeddings
2022-05-11 11:14:28,981 - BERTopic - Reduced dimensionality
2022-05-11 11:14:28,996 - BERTopic - Clustered reduced embeddings


IndexError: ignored

In [9]:
start_time = time.time()
iter_version = "BERTopic_SPECTER_0511"


for i in range(20,len(files)):
# for i in range(1):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]
  # beam_name= "Beamline_8.3.2.xls"
  # beam_name= "Beamline_7.3.3.xls"
  

  df = pd.read_table(path + files[i], on_bad_lines='skip')
  # df = pd.read_table(path + beam_name, on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input_new(df)

  # remove stopwords
  input_data['combined'] = input_data['combined'].apply(remove_stop_df)

  # lemmatize
  input_data['combined'] = input_data['combined'].apply(lemma_spacy)
  # df_lemma = input_data['combined']

  # remove punct
  input_data['combined'] = input_data['combined'].apply(remove_punct_df)

   # two iterations - one for unigram, other for bigram
  for j in range(2):

    print("******For ngram {}*****:".format(j+1))

    sentence_model = SentenceTransformer('allenai-specter')
    topic_model = BERTopic(embedding_model=sentence_model,nr_topics='auto',verbose=True,n_gram_range=(j+1,j+1))

    # topic_model = BERTopic(nr_topics='auto',verbose=True,n_gram_range=(i+1,i+1))
    list_text = input_data['combined'].to_list()

    try:
      topics, probs = topic_model.fit_transform(list_text)

      years = input_data['pub_year'].to_list()

      # defining path
      base_path = '/content/drive/MyDrive/NLP/' + iter_version + "_ngram_" + str(j+1) + "/" 
      model_path = base_path + beam_name + "/"
      create_path_if_not_exists(model_path)

      # save model as pickle file
      file_path = model_path + "model" + beam_name + ".pkl"
      joblib.dump(topic_model, file_path) 

      # dynamic topic modeling
      topics_over_time = topic_model.topics_over_time(list_text, topics, years)
      fig_time = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)
      fig_name_png = model_path + "topic_time" + beam_name  +".png"
      fig_time.write_image(fig_name_png)
      fig_name_html = model_path + "topic_time" + beam_name  +".html"
      fig_time.write_html(fig_name_html)

      # visualize barchart of topics as html
      fig = topic_model.visualize_barchart(top_n_topics = len(topic_model.topics))
      fig_name = model_path + "bar_chart" + beam_name  +".html"
      fig_name_png = model_path + "bar_chart" + beam_name  +".png"

      fig.write_html(fig_name)
      fig.write_image(fig_name_png)


      # save as excel
      # df_info = pd.DataFrame(model.get_topic_info())
      excel_name = model_path + "Topic_Results.xlsx"
      # df_info.to_excel(excel_name,sheet_name="topic_info")

      df_topics = pd.DataFrame(topic_model.topics)
      df_topics.to_excel(excel_name,sheet_name="topic_words")

    except:
      print("*****Some Error*****:")

    print("File: {} done ".format(j+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))



File: 21 Beamline_4.0.3.1.xls
******For ngram 1*****:


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2022-05-11 11:23:36,899 - BERTopic - Transformed documents to Embeddings
2022-05-11 11:23:40,288 - BERTopic - Reduced dimensionality
2022-05-11 11:23:40,301 - BERTopic - Clustered reduced embeddings


*****Some Error*****:
File: 1 done 
******For ngram 2*****:


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

...

2022-05-11 11:58:04,991 - BERTopic - Transformed documents to Embeddings
2022-05-11 11:58:08,882 - BERTopic - Reduced dimensionality
2022-05-11 11:58:08,909 - BERTopic - Clustered reduced embeddings
2022-05-11 11:58:09,161 - BERTopic - Reduced number of topics from 6 to 6
16it [00:00, 59.63it/s]


File: 1 done 
******For ngram 2*****:


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

2022-05-11 11:58:19,582 - BERTopic - Transformed documents to Embeddings
2022-05-11 11:58:25,470 - BERTopic - Reduced dimensionality
2022-05-11 11:58:25,493 - BERTopic - Clustered reduced embeddings
2022-05-11 11:58:25,859 - BERTopic - Reduced number of topics from 5 to 5
16it [00:00, 31.71it/s]


File: 2 done 
Total time taken (mins):  35
